<h1 class='font-effect-3d' style='font-family:Ewert; color:#ff355e'> Code Modules, Styling, and Helpful Functions</h1>

In [1]:
%%html
<style> 
@import url('https://fonts.googleapis.com/css?family=Ewert|Roboto&effect=3d'); 
h1,h3 {text-shadow:2px 2px 2px #ff355e;} 
span {font-family:Roboto; color:midnightblue; text-shadow:4px 4px 4px #aaa;}
div.output_prompt {color:#ff00cc;}
div.input_prompt {color:#ff355e;} 
div.output_area pre,div.output_subarea {font-family:Roboto; font-size:15px; color:#ff355e;}
div.output_stderr pre {background-color:ghostwhite;}
</style>

In [2]:
import warnings; warnings.filterwarnings('ignore')
import os,h5py
import pandas as pd,numpy as np,xgboost as xgb
import pylab as pl,seaborn as sn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from keras.datasets import mnist
from keras.utils import to_categorical
path1='../input/digit-recognizer/'
path2='../input/classification-of-handwritten-letters/'

Using TensorFlow backend.


In [3]:
def ohe(x): 
    return OneHotEncoder(categories='auto').fit(x.reshape(-1,1))\
           .transform(x.reshape(-1,1)).toarray().astype('int64')
def tts(X,y): 
    x_train,x_test,y_train,y_test=\
    train_test_split(X,y,test_size=.2,random_state=1)
    n=int(len(x_test)/2)
    x_valid,y_valid=x_test[:n],y_test[:n]
    x_test,y_test=x_test[n:],y_test[n:]
    return x_train,x_valid,x_test,y_train,y_valid,y_test

<h1 class='font-effect-3d' style='font-family:Ewert; color:#ff355e'> Data Sets</h1>

In [4]:
digits=pd.read_csv(path1+'train.csv')
digit_labels=digits['label'].values.astype('int16')
#cdigit_labels=ohe(digit_labels)
digit_images=digits.drop(labels=["label"],axis=1) 
digit_images=digit_images.values.astype('float32')/255
digit_images=(digit_images-.5)*2
[digit_labels.shape,digit_images.shape]

[(42000,), (42000, 784)]

In [5]:
x_train1,x_valid1,x_test1,\
y_train1,y_valid1,y_test1=tts(digit_images,digit_labels)
#y_train1=np.array([np.argmax(y) for y in y_train1])
#y_valid1=np.array([np.argmax(y) for y in y_valid1])
#y_test1=np.array([np.argmax(y) for y in y_test1])
[el.shape for el in [x_train1,x_valid1,x_test1,
                     y_train1,y_valid1,y_test1]]


[(33600, 784), (4200, 784), (4200, 784), (33600,), (4200,), (4200,)]

In [6]:
(x_train4,y_train4),(x_test4,y_test4)=mnist.load_data()
n=int(len(x_test4)/2)
x_valid4,y_valid4=x_test4[:n],y_test4[:n]
x_test4,y_test4=x_test4[n:],y_test4[n:]
#cy_train4=to_categorical(y_train4,10)
#cy_valid4=to_categorical(y_valid4,10)
#cy_test4=to_categorical(y_test4,10)
x_train4=x_train4.reshape(-1,784)
x_test4=x_test4.reshape(-1,784)
x_valid4=x_valid4.reshape(-1,784)
[el.shape for el in [x_train4,x_valid4,x_test4,
                     y_train4,y_valid4,y_test4]]

11493376/11490434 [==============================] - 1s 0us/step


[(60000, 784), (5000, 784), (5000, 784), (60000,), (5000,), (5000,)]

In [7]:
f=h5py.File(path2+'LetterColorImages2.h5','r')
keys=list(f.keys())
letters=u'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
letter_labels=np.array(f[keys[2]])
letter_images=np.array(f[keys[1]])/255
#cletter_labels=ohe(letter_labels)
gray_letter_images=np.dot(letter_images[...,:3],
                          [.299,.587,.114])
gray_letter_images=gray_letter_images.reshape(-1,32,32,1)
[letter_labels.shape,
 letter_images.shape,gray_letter_images.shape]

[(5940,), (5940, 32, 32, 3), (5940, 32, 32, 1)]

In [8]:
x_train2,x_valid2,x_test2,\
y_train2,y_valid2,y_test2=tts(letter_images,letter_labels)
x_train3,x_valid3,x_test3,\
y_train3,y_valid3,y_test3=tts(gray_letter_images,letter_labels)
#ny_train2=np.array([np.argmax(y) for y in y_train2])
#ny_valid2=np.array([np.argmax(y) for y in y_valid2])
#ny_test2=np.array([np.argmax(y) for y in y_test2])
#ny_train3=np.array([np.argmax(y) for y in y_train3])
#ny_valid3=np.array([np.argmax(y) for y in y_valid3])
#ny_test3=np.array([np.argmax(y) for y in y_test3])
[el.shape for el in [x_train2,x_valid2,x_test2,
                     y_train2,y_valid2,y_test2,
                     x_train3,x_valid3,x_test3,
                     y_train3,y_valid3,y_test3]]

[(4752, 32, 32, 3),
 (594, 32, 32, 3),
 (594, 32, 32, 3),
 (4752,),
 (594,),
 (594,),
 (4752, 32, 32, 1),
 (594, 32, 32, 1),
 (594, 32, 32, 1),
 (4752,),
 (594,),
 (594,)]

<h1 class='font-effect-3d' style='font-family:Ewert; color:#ff355e'>XGB Classifiers</h1>
### Digits

In [9]:
params={'objective':'multi:softprob','verbosity':2,
        'random_state':42,'num_class':10,
        'n_estimators':784,'learning_rate':.3,'max_depth':11,
        'tree_method':'gpu_hist','predictor':'gpu_predictor'}
clf=xgb.XGBClassifier(**params)
clf.fit(x_train1,y_train1,eval_metric='mlogloss',
        eval_set=[(x_train1,y_train1),(x_valid1,y_valid1)])

[0]	validation_0-mlogloss:1.18529	validation_1-mlogloss:1.26445
[1]	validation_0-mlogloss:0.84093	validation_1-mlogloss:0.95293
[2]	validation_0-mlogloss:0.62241	validation_1-mlogloss:0.75444
[3]	validation_0-mlogloss:0.46804	validation_1-mlogloss:0.61531
[4]	validation_0-mlogloss:0.35693	validation_1-mlogloss:0.51305
[5]	validation_0-mlogloss:0.27430	validation_1-mlogloss:0.43487
[6]	validation_0-mlogloss:0.21239	validation_1-mlogloss:0.37460
[7]	validation_0-mlogloss:0.16550	validation_1-mlogloss:0.32820
[8]	validation_0-mlogloss:0.12966	validation_1-mlogloss:0.29197
[9]	validation_0-mlogloss:0.10229	validation_1-mlogloss:0.26158
[10]	validation_0-mlogloss:0.08118	validation_1-mlogloss:0.23778
[11]	validation_0-mlogloss:0.06508	validation_1-mlogloss:0.21868
[12]	validation_0-mlogloss:0.05237	validation_1-mlogloss:0.20225
[13]	validation_0-mlogloss:0.04248	validation_1-mlogloss:0.18951
[14]	validation_0-mlogloss:0.03487	validation_1-mlogloss:0.17841
[15]	validation_0-mlogloss:0.02883	

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.3, max_delta_step=0, max_depth=11,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=784, n_jobs=0, num_class=10, num_parallel_tree=1,
              objective='multi:softprob', predictor='gpu_predictor',
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='gpu_hist', validate_parameters=False,
              verbosity=2)

In [10]:
print(clf.evals_result())
y_xgb_train1=clf.predict(x_train1)
y_xgb_valid1=clf.predict(x_valid1)
y_xgb_test1=clf.predict(x_test1)
for [y,py] in [[y_train1,y_xgb_train1],
               [y_valid1,y_xgb_valid1],
               [y_test1,y_xgb_test1]]:
    print(metrics.accuracy_score(y,py))
    print(metrics.confusion_matrix(y,py))

{'validation_0': {'mlogloss': [1.185291, 0.840935, 0.622409, 0.468036, 0.356928, 0.274303, 0.212389, 0.1655, 0.129658, 0.10229, 0.081184, 0.065078, 0.052365, 0.04248, 0.034871, 0.028834, 0.024158, 0.020395, 0.017371, 0.014891, 0.012951, 0.011348, 0.010029, 0.008916, 0.008007, 0.007215, 0.006547, 0.005994, 0.005477, 0.005053, 0.004681, 0.004329, 0.004039, 0.003781, 0.003567, 0.003357, 0.003176, 0.003014, 0.002865, 0.00273, 0.002613, 0.002503, 0.002399, 0.002307, 0.002216, 0.002138, 0.00206, 0.001988, 0.001919, 0.001863, 0.001807, 0.001756, 0.001707, 0.001659, 0.001614, 0.001573, 0.001536, 0.001496, 0.001461, 0.001429, 0.001399, 0.001368, 0.00134, 0.001312, 0.001287, 0.001263, 0.001238, 0.001216, 0.001193, 0.001173, 0.001154, 0.001134, 0.001115, 0.001098, 0.00108, 0.001063, 0.001048, 0.001033, 0.001019, 0.001006, 0.000992, 0.000979, 0.000966, 0.000952, 0.00094, 0.000929, 0.000916, 0.000906, 0.000894, 0.000884, 0.000873, 0.000863, 0.000853, 0.000841, 0.000833, 0.000824, 0.000815, 0.000807

In [11]:
params={'objective':'multi:softprob','verbosity':2,
        'random_state':42,'num_class':10,
        'n_estimators':784,'learning_rate':.3,'max_depth':11,
        'tree_method':'gpu_hist','predictor':'gpu_predictor'}
clf4=xgb.XGBClassifier(**params)
clf4.fit(x_train4,y_train4,eval_metric='mlogloss',
         eval_set=[(x_train4,y_train4),(x_valid4,y_valid4)])

[0]	validation_0-mlogloss:1.17564	validation_1-mlogloss:1.27876
[1]	validation_0-mlogloss:0.83419	validation_1-mlogloss:0.96850
[2]	validation_0-mlogloss:0.61577	validation_1-mlogloss:0.76589
[3]	validation_0-mlogloss:0.46464	validation_1-mlogloss:0.62480
[4]	validation_0-mlogloss:0.35570	validation_1-mlogloss:0.52300
[5]	validation_0-mlogloss:0.27438	validation_1-mlogloss:0.44521
[6]	validation_0-mlogloss:0.21249	validation_1-mlogloss:0.38309
[7]	validation_0-mlogloss:0.16586	validation_1-mlogloss:0.33570
[8]	validation_0-mlogloss:0.13005	validation_1-mlogloss:0.29811
[9]	validation_0-mlogloss:0.10274	validation_1-mlogloss:0.26943
[10]	validation_0-mlogloss:0.08147	validation_1-mlogloss:0.24670
[11]	validation_0-mlogloss:0.06498	validation_1-mlogloss:0.22709
[12]	validation_0-mlogloss:0.05225	validation_1-mlogloss:0.21215
[13]	validation_0-mlogloss:0.04226	validation_1-mlogloss:0.19892
[14]	validation_0-mlogloss:0.03463	validation_1-mlogloss:0.18859
[15]	validation_0-mlogloss:0.02873	

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.3, max_delta_step=0, max_depth=11,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=784, n_jobs=0, num_class=10, num_parallel_tree=1,
              objective='multi:softprob', predictor='gpu_predictor',
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='gpu_hist', validate_parameters=False,
              verbosity=2)

In [12]:
print(clf4.evals_result())
y_xgb_train4=clf4.predict(x_train4)
y_xgb_valid4=clf4.predict(x_valid4)
y_xgb_test4=clf4.predict(x_test4)
for [y,py] in [[y_train4,y_xgb_train4],
               [y_valid4,y_xgb_valid4],
               [y_test4,y_xgb_test4]]:
    print(metrics.accuracy_score(y,py))
    print(metrics.confusion_matrix(y,py))

{'validation_0': {'mlogloss': [1.175639, 0.83419, 0.615774, 0.464643, 0.355705, 0.27438, 0.212494, 0.165857, 0.13005, 0.102736, 0.081466, 0.064983, 0.052249, 0.042263, 0.034629, 0.028732, 0.0239, 0.020137, 0.017056, 0.014607, 0.012558, 0.010897, 0.009585, 0.008471, 0.007514, 0.006719, 0.006064, 0.005502, 0.005006, 0.004586, 0.004225, 0.003905, 0.003629, 0.003375, 0.003159, 0.002973, 0.002793, 0.002628, 0.002487, 0.002357, 0.002247, 0.002139, 0.002041, 0.001956, 0.001873, 0.0018, 0.001732, 0.00167, 0.001612, 0.001556, 0.001508, 0.001461, 0.001418, 0.001378, 0.001336, 0.001298, 0.001263, 0.00123, 0.0012, 0.001171, 0.001142, 0.001115, 0.00109, 0.001067, 0.001044, 0.001022, 0.001002, 0.000982, 0.000961, 0.000943, 0.000925, 0.000909, 0.000891, 0.000876, 0.000862, 0.000848, 0.000834, 0.000821, 0.000807, 0.000795, 0.000783, 0.000771, 0.000759, 0.000748, 0.000737, 0.000725, 0.000716, 0.000707, 0.000698, 0.000689, 0.000681, 0.000673, 0.000665, 0.000656, 0.000649, 0.000641, 0.000634, 0.000627, 0

### Letters
Color Images

In [13]:
clf2=xgb.XGBClassifier(objective="multi:softprob",verbosity=2,
                       random_state=42,learning_rate=.05,
                       max_depth=16,n_estimators=256,num_class=33,
                       tree_method='gpu_hist',predictor='gpu_predictor')
clf2.fit(x_train2.reshape(-1,32*32*3),y_train2,eval_metric='mlogloss',
         eval_set=[(x_train2.reshape(-1,32*32*3),y_train2),
                   (x_valid2.reshape(-1,32*32*3),y_valid2)])

[0]	validation_0-mlogloss:3.29736	validation_1-mlogloss:3.41887
[1]	validation_0-mlogloss:3.11329	validation_1-mlogloss:3.35751
[2]	validation_0-mlogloss:2.94204	validation_1-mlogloss:3.30280
[3]	validation_0-mlogloss:2.78567	validation_1-mlogloss:3.25238
[4]	validation_0-mlogloss:2.64035	validation_1-mlogloss:3.20273
[5]	validation_0-mlogloss:2.50568	validation_1-mlogloss:3.16048
[6]	validation_0-mlogloss:2.38213	validation_1-mlogloss:3.12638
[7]	validation_0-mlogloss:2.26553	validation_1-mlogloss:3.09002
[8]	validation_0-mlogloss:2.15774	validation_1-mlogloss:3.06025
[9]	validation_0-mlogloss:2.05773	validation_1-mlogloss:3.03404
[10]	validation_0-mlogloss:1.96265	validation_1-mlogloss:3.00495
[11]	validation_0-mlogloss:1.87294	validation_1-mlogloss:2.97866
[12]	validation_0-mlogloss:1.78900	validation_1-mlogloss:2.95335
[13]	validation_0-mlogloss:1.70947	validation_1-mlogloss:2.93267
[14]	validation_0-mlogloss:1.63445	validation_1-mlogloss:2.91363
[15]	validation_0-mlogloss:1.56386	

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.05, max_delta_step=0, max_depth=16,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=256, n_jobs=0, num_class=33, num_parallel_tree=1,
              objective='multi:softprob', predictor='gpu_predictor',
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='gpu_hist', validate_parameters=False,
              verbosity=2)

In [14]:
print(clf2.evals_result())
y_xgb_train2=clf2.predict(x_train2.reshape(-1,32*32*3))
y_xgb_valid2=clf2.predict(x_valid2.reshape(-1,32*32*3))
y_xgb_test2=clf2.predict(x_test2.reshape(-1,32*32*3))
for [y,py] in [[y_train2,y_xgb_train2],
               [y_valid2,y_xgb_valid2],
               [y_test2,y_xgb_test2]]:
    print(metrics.accuracy_score(y,py))
    print(metrics.confusion_matrix(y,py))

{'validation_0': {'mlogloss': [3.297358, 3.113295, 2.942044, 2.785674, 2.640347, 2.505676, 2.382133, 2.265532, 2.157744, 2.057732, 1.962654, 1.872937, 1.788997, 1.709468, 1.634455, 1.563863, 1.497194, 1.433852, 1.373845, 1.317106, 1.263074, 1.212075, 1.163203, 1.117002, 1.073102, 1.031588, 0.991822, 0.954024, 0.917755, 0.88325, 0.85024, 0.818776, 0.788607, 0.759677, 0.732207, 0.705587, 0.680418, 0.656094, 0.633124, 0.610893, 0.589584, 0.569247, 0.549678, 0.531007, 0.513093, 0.495842, 0.479298, 0.463359, 0.448111, 0.433405, 0.419349, 0.405829, 0.392889, 0.380476, 0.368433, 0.356958, 0.345786, 0.335115, 0.324827, 0.314973, 0.305504, 0.296366, 0.287644, 0.279183, 0.271029, 0.263284, 0.255675, 0.248384, 0.241352, 0.234578, 0.228126, 0.221871, 0.215836, 0.210035, 0.204477, 0.199025, 0.193829, 0.188824, 0.18399, 0.179297, 0.174786, 0.170465, 0.16624, 0.162194, 0.158274, 0.154498, 0.150807, 0.147276, 0.143867, 0.140538, 0.137357, 0.134255, 0.131254, 0.128332, 0.125544, 0.12283, 0.12021, 0.117

Grayscaled Images

In [15]:
clf3=xgb.XGBClassifier(objective="multi:softprob",verbosity=2,
                       random_state=42,max_depth=16,num_class=33,
                       tree_method='gpu_hist',predictor='gpu_predictor')
clf3.fit(x_train3.reshape(-1,32*32),y_train3,eval_metric='mlogloss',
         eval_set=[(x_train3.reshape(-1,32*32),y_train3),
                   (x_valid3.reshape(-1,32*32),y_valid3)])

[0]	validation_0-mlogloss:2.48175	validation_1-mlogloss:3.23650
[1]	validation_0-mlogloss:1.84407	validation_1-mlogloss:3.10283
[2]	validation_0-mlogloss:1.40063	validation_1-mlogloss:3.01107
[3]	validation_0-mlogloss:1.08727	validation_1-mlogloss:2.93749
[4]	validation_0-mlogloss:0.85794	validation_1-mlogloss:2.88269
[5]	validation_0-mlogloss:0.68656	validation_1-mlogloss:2.83568
[6]	validation_0-mlogloss:0.55526	validation_1-mlogloss:2.79129
[7]	validation_0-mlogloss:0.45438	validation_1-mlogloss:2.76242
[8]	validation_0-mlogloss:0.37502	validation_1-mlogloss:2.73351
[9]	validation_0-mlogloss:0.31213	validation_1-mlogloss:2.70032
[10]	validation_0-mlogloss:0.26249	validation_1-mlogloss:2.67600
[11]	validation_0-mlogloss:0.22281	validation_1-mlogloss:2.65231
[12]	validation_0-mlogloss:0.19099	validation_1-mlogloss:2.63513
[13]	validation_0-mlogloss:0.16536	validation_1-mlogloss:2.62001
[14]	validation_0-mlogloss:0.14460	validation_1-mlogloss:2.60603
[15]	validation_0-mlogloss:0.12746	

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=16,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_class=33, num_parallel_tree=1,
              objective='multi:softprob', predictor='gpu_predictor',
              random_state=42, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='gpu_hist', validate_parameters=False,
              verbosity=2)

In [16]:
print(clf3.evals_result())
y_xgb_train3=clf3.predict(x_train3.reshape(-1,32*32))
y_xgb_valid3=clf3.predict(x_valid3.reshape(-1,32*32))
y_xgb_test3=clf3.predict(x_test3.reshape(-1,32*32))
for [y,py] in [[y_train3,y_xgb_train3],
               [y_valid3,y_xgb_valid3],
               [y_test3,y_xgb_test3]]:
    print(metrics.accuracy_score(y,py))
    print(metrics.confusion_matrix(y,py))

{'validation_0': {'mlogloss': [2.481753, 1.844066, 1.400633, 1.087274, 0.857941, 0.686564, 0.555255, 0.454379, 0.375024, 0.312127, 0.26249, 0.222813, 0.190988, 0.165356, 0.144603, 0.127455, 0.113124, 0.101207, 0.09123, 0.082859, 0.075619, 0.069386, 0.064028, 0.059409, 0.055296, 0.051727, 0.048591, 0.045841, 0.043359, 0.04114, 0.039156, 0.037363, 0.035744, 0.034259, 0.032902, 0.031688, 0.030549, 0.02952, 0.028545, 0.027638, 0.026808, 0.026029, 0.025305, 0.024636, 0.023997, 0.02341, 0.022817, 0.022288, 0.021785, 0.021312, 0.020865, 0.020438, 0.020036, 0.019651, 0.019292, 0.018942, 0.018617, 0.018301, 0.018009, 0.01772, 0.017445, 0.017188, 0.016942, 0.01669, 0.016457, 0.01623, 0.016017, 0.015815, 0.015608, 0.015411, 0.015218, 0.015036, 0.014862, 0.014694, 0.014538, 0.014378, 0.014217, 0.014067, 0.013923, 0.013781, 0.013643, 0.013512, 0.013379, 0.013252, 0.01313, 0.013011, 0.012897, 0.012784, 0.012674, 0.012576, 0.012474, 0.012375, 0.012272, 0.012176, 0.012084, 0.01199, 0.011904, 0.011813,